<a href="https://colab.research.google.com/github/nidjosep/student-feedback-analysis/blob/master/models/model-source/Model_2_Actionable_Insights_from_Student_Feedback_SA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import ssl
# from google.colab import drive

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Download NLTK stopwords
nltk.download('stopwords')

# Load stop words
stop_words = set(stopwords.words('english'))

# Function to remove stop words
def remove_stop_words(text):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

# Load the dataset
df = pd.read_csv('/Users/ashwantmanikoth/Desktop/UNB/ML_workFlow/Emotion_final.csv')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ashwantmanikoth/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout, Bidirectional
from tensorflow.keras import regularizers  # Import the regularizers module
from tensorflow.keras.optimizers import Adam  # Import Adam optimizer
from tensorflow.keras.callbacks import EarlyStopping  # Import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd


texts = df['Review']
labels = df['Label']

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and return encoded labels
encoded_labels = label_encoder.fit_transform(labels)

# To see the mapping of string labels to integers
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping: ", label_mapping)

# Hyperparameters
vocab_size = 10000  # Number of unique words in the vocabulary
embedding_dim = 100  # Number of dimensions for each word vector
max_length = 100    # Max length of each input sequence
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"   # Token for out-of-vocabulary words

# Tokenize the text
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(padded, encoded_labels, test_size=0.2, random_state=42)

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(GRU(64, return_sequences=True)),
    Bidirectional(GRU(32)),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

# Using Adam optimizer with a different learning rate
optimizer = Adam(learning_rate=0.0005)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(train_texts, train_labels, epochs=20, batch_size=64, validation_data=(val_texts, val_labels), callbacks=[early_stopping])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


Label Mapping:  {'anger': 0, 'fear': 1, 'happy': 2, 'love': 3, 'sadness': 4, 'surprise': 5}


Epoch 1/20
269/269 [==============================] - 29s 96ms/step - loss: 1.8177 - accuracy: 0.4082 - val_loss: 1.2638 - val_accuracy: 0.5813
Epoch 2/20
269/269 [==============================] - 25s 92ms/step - loss: 1.1017 - accuracy: 0.6189 - val_loss: 0.9451 - val_accuracy: 0.6750
Epoch 3/20
269/269 [==============================] - 24s 88ms/step - loss: 0.8245 - accuracy: 0.7012 - val_loss: 0.7982 - val_accuracy: 0.7074
Epoch 4/20
269/269 [==============================] - 23s 87ms/step - loss: 0.6747 - accuracy: 0.7435 - val_loss: 0.7103 - val_accuracy: 0.7444
Epoch 5/20
269/269 [==============================] - 24s 91ms/step - loss: 0.5491 - accuracy: 0.7935 - val_loss: 0.6542 - val_accuracy: 0.7724
Epoch 6/20
269/269 [==============================] - 23s 87ms/step - loss: 0.4620 - accuracy: 0.8392 - val_loss: 0.5836 - val_accuracy: 0.8304
Epoch 7/20
269/269 [==============================] - 24s 90ms/step - loss: 0.3807 - accuracy: 0.8946 - val_loss: 0.5176 - val_accuracy:

In [ ]:
def predict_sentiment(text):
    # Tokenize and pad the text
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    # Predict
    prediction = model.predict(padded_sequence)
    sentiment = label_encoder.inverse_transform([np.argmax(prediction)])[0]

    return sentiment

# Example usage
sentence = "Professor is nice"  # Replace with your sentence
predicted_sentiment = predict_sentiment(sentence)
print(f"The predicted sentiment is: {predicted_sentiment}")


1/1 [==============================] - 0s 13ms/step
The predicted sentiment is: fear


In [ ]:
import json
def read_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Function to write JSON file
def write_json(data, file_path):
    with open(file_path, 'w') as file:
        json.dump(data, file)


def remove_stop_words_from_list(texts):
    # Define a list of words to whitelist
    whitelist = []
    processed_texts = []
    for text in texts:
        processed_text = ' '.join([word for word in text.split() if word.lower() not in stop_words or word.lower() in whitelist])
        processed_texts.append(processed_text)
    return processed_texts


# Read test data
test_data_dict = read_json('/Users/ashwantmanikoth/Desktop/UNB/ML_workFlow/test_dataset_all_models.json')  # Make sure this path is correct

# Extract messages from the test data
test_messages = list(test_data_dict.values())
test_messages = remove_stop_words_from_list(test_messages)
# Preprocess test data
X_test_seq = tokenizer.texts_to_sequences(test_messages)
X_test_pad = pad_sequences(X_test_seq, maxlen=150)  # Adjust maxlen if needed

# Make predictions
predictions = model.predict(X_test_pad)
predicted_labels = label_encoder.inverse_transform([np.argmax(p) for p in predictions])

# Prepare data for JSON output
output_predictions = {id: str(label) for id, label in zip(test_data_dict.keys(), predicted_labels)}
print(output_predictions)
# Write predictions to JSON
write_json(output_predictions, '/Users/ashwantmanikoth/Desktop/UNB/ML_workFlow/ratings.json')

print("Predictions saved to predictions.json")


4/4 [==============================] - 0s 17ms/step
{'1': 'surprise', '2': 'surprise', '3': 'love', '4': 'surprise', '5': 'surprise', '6': 'fear', '7': 'fear', '8': 'surprise', '9': 'fear', '10': 'surprise', '11': 'love', '12': 'surprise', '13': 'love', '14': 'fear', '15': 'surprise', '16': 'love', '17': 'surprise', '18': 'love', '19': 'surprise', '20': 'love', '21': 'love', '22': 'surprise', '23': 'surprise', '24': 'fear', '25': 'surprise', '26': 'love', '27': 'love', '28': 'surprise', '29': 'love', '30': 'fear', '31': 'surprise', '32': 'surprise', '33': 'love', '34': 'love', '35': 'surprise', '36': 'surprise', '37': 'surprise', '38': 'love', '39': 'surprise', '40': 'surprise', '41': 'surprise', '42': 'love', '43': 'love', '44': 'surprise', '45': 'surprise', '46': 'surprise', '47': 'love', '48': 'surprise', '49': 'love', '50': 'surprise', '51': 'surprise', '52': 'surprise', '53': 'surprise', '54': 'surprise', '55': 'love', '56': 'surprise', '57': 'love', '58': 'love', '59': 'love', '6